# SAT Solver Assignment
Group 69<br>
**Knowledge Representation**<br>
November 2024

****

## Preperations

*Everything we need to have before we can build the SAT Solver.*

**Imports** - Use this cell for all imports.

In [765]:
# import modules
import math
from tqdm import tqdm

**Data** - Import the needed data. The data needs to be in DIMACS format. Each clause in DIMACS CNF consists of:

- A series of positive or negative literals (numbers) that represent a Boolean variable or its negation.
- Each clause ends with 0.
- Each variable in DIMACS CNF is a positive integer, with unique integers representing each Boolean variable.

For Sudoku, we use a triplet `(r, c, n)`:
- `r` for the row (1 to 9)
- `c` for the column (1 to 9)
- `n` for the number (1 to 9)

In [766]:
# Geeft een file met 91 sudoku's terug, sudokus[0] is de eerste sudoku met len(sudokus[0]) == 81 (9x9=81 karakters)
with open('./top91.sdk.txt') as f:
    sudokus = f.read().splitlines()

*You are free to choose any programming language you fancy, but we must be able to run your SAT solver with the command SAT -Sn inputfile , for example: SAT -S2 sudoku_nr_10 , where SAT is the (compulsory) name of your program, n=1 for the basic DP and n=2 or 3 for your two other strategies, and the input file is the concatenation of all required input clauses (in your case: sudoku rules + given puzzle).*

****

## Part 1 - Building the SAT Solver

**The Rules of Sudoku**
1. Each number (1-9) must appear exactly once in each row.
2. Each number must appear exactly once in each column.
3. Each number must appear exactly once in each 3x3 subgrid.

**The Data of Sudoku in DIMACS**

In [767]:
def load_rules(filename: str) -> list[list[int]]:
    with open(filename) as f:
        raw_rules = f.read().splitlines()

    rules = [list(map(int, clause.split()))[:-1] for clause in raw_rules[1:]]
    return rules

def load_sudoku(filename: str) -> list[list[list[int | str]]]:
    with open(filename) as f:
        sudoku = f.read().splitlines()

    puzzles: list[list[list[int | str]]] = []
    
    puzzledim: int = math.sqrt(len(sudoku[0]))
    assert puzzledim.is_integer(), "The sudoku is not square"
    puzzledim = int(puzzledim)

    for puzzle in sudoku:
        newpuzzle = []
        for r in range(puzzledim):
            row = []
            for c in range(puzzledim):
                val = puzzle[r * puzzledim + c]
                val = int(val) if val != '.' else 0 # convert to int if we know the value
                row.append(val)
            newpuzzle.append(row)
        puzzles.append(newpuzzle)
    return puzzles

**Functions to Assign Obvious Values** - Fill in the single and pure literal clauses.

In [768]:
def single_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]] | None:
    """
    If a clause has only one literal, assigns it to satisfy the clause, removes satisfied clauses, 
    and removes negated literals from the remaining clauses. Repeats until no single-literal 
    clauses remain or an unsatisfiable clause (empty clause) is found.

    Args:
        clauses (list[list[int]]): List of clauses, each a list of integers (literals).
        assignment (dict[int, bool]): Current variable assignments, aka the sudoku.

    Returns:
        list[list[int]] | None: Updated list of clauses, or None if unsatisfiable.
    """
    while any(len(clause) == 1 for clause in clauses):
        for clause in clauses:
            if len(clause) == 1:
                literal = clause[0]
                assignment[abs(literal)] = literal > 0
                new_clauses = []
                for c in clauses:
                    if literal in c:
                        continue
                    updated_clause = [l for l in c if l != -literal]
                    if not updated_clause:
                        return None
                    new_clauses.append(updated_clause)
                clauses = new_clauses
                break
    return clauses


def pure_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]]:
    """
    Assigns values to pure literals (literals with no negation in any clause) and updates clauses.

    Args:
        clauses (List[List[int]]): List of clauses, each as a list of literals.
        assignment (Dict[int, bool]): Dictionary of current variable assignments, aka the sudoku.

    Returns:
        List[List[int]]: Updated list of clauses with pure literals removed.
    """
    literals = {lit for clause in clauses for lit in clause}
    pure_literals = [l for l in literals if -l not in literals]
    
    for literal in pure_literals:
        assignment[abs(literal)] = literal > 0
        clauses = [clause for clause in clauses if literal not in clause]
    
    return clauses


**DPLL Algorithm** - DPLL recursive algorithm function,  without heuristics, to perform the fucntions above and recursively gives values to literals.

In [770]:
def dpll(rules: list[list[int]], assignment: dict[int, bool], size, pbar: tqdm, history: list[int] = [0]) -> dict[int, bool] | None:
    """
    Solves a set of clauses using the DPLL algorithm.

    Args:
        clauses (list[list[int]]): A list of clauses, each as a list of literals.
        assignment (dict[int, bool]): Current variable assignments, aka the sudoku.
        pbar (tqdm): Progress bar for the solver.
        history (list[int]): List of integers representing the path of the solver.

    Returns:
        dict[int, bool] | None: Satisfying assignment if one exists, otherwise None.
    """
    print("Start DPLL with history:", history)

    # Simplify the clauses using unit propagation and pure literal elimination
    clauses = single_literal(rules, assignment)
    if clauses is None:
        return None  # conflict found
    if not clauses:
        return assignment  # all clauses satisfied

    clauses = pure_literal(clauses, assignment)
    if not clauses:
        return assignment  # all clauses satisfied
    
    # Select a variable to assign (choose from the unknown cells)
    unassigned_vars = [var for clause in clauses for var in clause if var not in assignment]
    print(unassigned_vars)
    if not unassigned_vars:
        print("No unassigned variables left")
        return None  # No unassigned variables left, puzzle is inconsistent

    literal = unassigned_vars[0]  # Pick the first unassigned literal
    print(f"Trying literal: {literal} with assignment {assignment}")

    # Try assigning True to the literal
    assignment[literal] = True
    updated_clauses = [c for c in clauses if literal not in c]  # Remove clauses satisfied by the literal
    pbar.update(1)
    print(f"Assigning {literal} = True")
    result = dpll(updated_clauses, assignment.copy(), size, pbar, history + [literal])
    if result:
        return result  # If assignment works, return it

    # If assigning True doesn't work, try assigning False to the literal
    assignment[literal] = False
    updated_clauses = [c for c in clauses if -literal not in c]  # Remove clauses satisfied by the negated literal
    pbar.update(1)
    print(f"Assigning {literal} = False")
    result = dpll(updated_clauses, assignment.copy(), size, pbar, history + [-literal])
    if result:
        return result  # If assignment works, return it

    print(f"No solution for {literal}, backtracking...")
    return None  # No solution found for this branch



*give (2)+(3) as input to (1) and return the solution to the given puzzle. This output should again be a DIMACS file, but containing only the truth assignment to all variables (729 for Sudoku, different for other SAT problems). If your input file is called 'filename', then make sure your outputfile is called 'filename.out'. If there is no solution (inconsistent problem), the output can be an empty file. If there are multiple solutions (eg. non-proper Sudoku) you only need to return a single solution.*

**Heuristics** - Two different heuristics of our choice

In [771]:
# heuristic one

In [772]:
# heuristic two

**SAT Solver** - Solves the Sudoku

Extra information about the code:
Each cell in a Sudoku puzzle can be represented by a unique variable based on its row, column, and possible number. The encoding formula for each variable is:

`variable=(i+1)×100+(j+1)×10+value`<br>

where:<br>

`i is the row index (0 to 8)`<br>
`j is the column index (0 to 8)`<br>
`value is the number in the cell (1 to 9)`<br>

This formula converts each cell and its possible values into unique numbers by combining the row, column, and value in a single integer. For example, if a cell at row 0, column 0 can be a 5, the encoded variable would be:<br>

`variable=(0+1)×100+(0+1)×10+5=115`<br>

In [773]:
def solve_sudoku(rules: list[list[int]], puzzle: list[list[int]], size) -> list[list[int]] | None:
    """
    Solves a Sudoku puzzle using given rules and an initial puzzle setup.

    Args:
        rules (list[list[int]]): A list of rules in CNF form for Sudoku constraints.
        puzzle (list[list[int]]): A grid of given size representing the puzzle, with `0` for empty cells.

    Returns:
        list[list[int]] | None: A solved Sudoku grid if a solution exists, otherwise None.
    """
    assignment = {} # the sudoku
    values2solve = {}
    cells_unknown = 0

    print(puzzle)
    for i, row in enumerate(puzzle):
        for j, value in enumerate(row):
            if value != 0:  # if 0, cell is empty, if not 0, it is a pre-filled cell
                # Make the value in row i, column j True if it is the value from the sudoku, all other numbers in that place False
                for k in range(size):
                    assignment[(i + 1) * 100 + (j + 1) * 10 + k + 1] = k + 1 == value
            else:
                for k in range(size):
                    values2solve[((i + 1) * 100 + (j + 1) * 10 + k + 1)] = k + 1 == value
                cells_unknown += 1
    print(f'The unknown values are: {values2solve}')
    # print(f'The known values are: {assignment}')

    with tqdm(total=2**cells_unknown, desc="Solving Sudoku") as pbar:
        solution = dpll(rules, values2solve, size, pbar)
    if not solution:
        return None
    
    # parse the solution back to a size grid
    # result = [[0] * size for _ in range(size)]
    # for var, value in solution.items():
    #     if value:
    #         i = (var // 100) - 1
    #         j = ((var // 10) % 10) - 1
    #         num = var % 10
    #         result[i][j] = num
    # print('check', result)
    # return result

    return solution

**Test Sudoku** - Try and see if it works

In [774]:
load_sudoku('./4x4_easy.txt')[1]

[[1, 2, 3, 4], [3, 4, 1, 0], [2, 1, 4, 3], [4, 3, 0, 1]]

In [775]:
# Load the Sudoku rules and the puzzle
filename = './sudoku-rules-4x4.txt'
rules = load_rules(filename)

# Load the puzzles from the file and select the second puzzle (index 1)
puzzle = load_sudoku('./4x4_easy.txt')[1]

size = 4  # Size of the grid (4x4 Sudoku)

# Solve the Sudoku puzzle
solution = solve_sudoku(rules, puzzle, size)

# If a solution is found, print the solved Sudoku grid
if solution:
    print("Solved Sudoku:", solution)
    for r in range(size):
        print(solution[r * size:(r + 1) * size])  # Print each row of the solution
else:
    print('No solution')

[[1, 2, 3, 4], [3, 4, 1, 0], [2, 1, 4, 3], [4, 3, 0, 1]]
The unknown values are: {241: False, 242: False, 243: False, 244: False, 431: False, 432: False, 433: False, 434: False}


Solving Sudoku: 42it [00:00, 5822.92it/s]            

Start DPLL with history: [0]
[111, 112, 113, 114, -111, -112, -111, -113, -111, -114, -112, -113, -112, -114, -113, -114, 121, 122, 123, 124, -121, -122, -121, -123, -121, -124, -122, -123, -122, -124, -123, -124, 131, 132, 133, 134, -131, -132, -131, -133, -131, -134, -132, -133, -132, -134, -133, -134, 141, 142, 143, 144, -141, -142, -141, -143, -141, -144, -142, -143, -142, -144, -143, -144, 211, 212, 213, 214, -211, -212, -211, -213, -211, -214, -212, -213, -212, -214, -213, -214, 221, 222, 223, 224, -221, -222, -221, -223, -221, -224, -222, -223, -222, -224, -223, -224, 231, 232, 233, 234, -231, -232, -231, -233, -231, -234, -232, -233, -232, -234, -233, -234, -241, -242, -241, -243, -241, -244, -242, -243, -242, -244, -243, -244, 311, 312, 313, 314, -311, -312, -311, -313, -311, -314, -312, -313, -312, -314, -313, -314, 321, 322, 323, 324, -321, -322, -321, -323, -321, -324, -322, -323, -322, -324, -323, -324, 331, 332, 333, 334, -331, -332, -331, -333, -331, -334, -332, -333, -3

KeyError: slice(0, 4, None)

****
## Part 2 - Experiment 

**Optional Hypothesis**: "Advancements in SAT solver techniques, tested and optimized using Sudoku puzzles as a structured benchmark, can significantly improve the efficiency of scheduling and resource allocation tasks in complex real-world applications, such as airline crew scheduling and automated exam timetabling. Specifically, Sudoku-based testing will highlight solvers' ability to handle strict mutual exclusivity and completeness constraints, which are essential for minimizing conflicts and optimizing resource usage in these applications."

This hypothesis aims to leverage Sudoku as a standardized problem to assess the performance and flexibility of SAT solvers in environments requiring rapid, constraint-based decision-making, potentially providing insights into solver improvements for more complex, real-world scheduling and resource optimization scenarios.